<img style="float: left;;" src='Figures/iteso.jpg' width="100" height="200"/></a>

# <center> <font color= #000047> Codificación con base en la media </font> </center>

In [ ]:
import pandas as pd
import numpy as np


data=pd.read_csv('temps.txt',sep='\s+',header=None) # Fuente: http://astro.iam.udg.mx/Estacion/NOAAPRMO.TXT
                                                         #         http://astro.iam.udg.mx/Estacion/NOAAMO.TXT
data.columns=['Temperatura máxima','Temperatura mínima','HDD','CDD','Rain']
d=['Sábado','Domingo','Lunes','Martes','Miércoles','Jueves','Viernes']
dias=[d[i%7] for i in range(len(data))]
catego=['Soleado','Mayormente soleado','Parcialmente nublado','Mayormente nublado','Nublado','Lluvioso']
prono=[catego[-1] if data['Rain'][i]>0 else catego[int(round(1.2*(data['HDD'][i]-data['CDD'][i])+2))] for i in range(len(data))]
df=pd.DataFrame(columns=['Días','Pronóstico','Temperatura máxima','Temperatura mínima'],index=range(len(data)))
df.iloc[:,2:4]=data.iloc[:,0:2]
df['Días']=dias
df['Pronóstico']=prono
df.head()

In [ ]:
#@title Codificando a traves del promedio
promedio=df.groupby('Pronóstico')['Temperatura mínima'].mean()
promedio

In [ ]:
df['Promedio']=df['Pronóstico'].map(promedio)
df.head()

In [ ]:
#@title Codificando a través de la codificación de Efectos
E=pd.get_dummies(df,columns=['Pronóstico'],drop_first=True,prefix='',prefix_sep='',)
E.head()

In [ ]:
CR,=set(df.Pronóstico.unique())-set(E.columns)
CR

In [ ]:
E.iloc[:,4:]=E.iloc[:,4:].astype('int')
idx_CR,=np.where(df.Pronóstico==CR)
E.iloc[idx_CR,4:]=-1
E.iloc[idx_CR]

In [ ]:
# Regresión lineal
from sklearn.linear_model import LinearRegression

In [ ]:
lin=LinearRegression()
lin.fit(E.iloc[:,4:],E['Temperatura mínima'])

In [ ]:
E.columns[4:]

In [ ]:
lin.coef_

In [ ]:
d_E={categorías:coeficientes for categorías,coeficientes in zip(E.columns[4:],lin.coef_)}
d_E

In [ ]:
d_E[CR]=lin.intercept_
d_E

In [ ]:
df['P_Efectos']=df.Pronóstico.map(d_E)
df.head()